In [1]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers

In [2]:
from tensorflow.python.client import device_lib
from tensorflow import keras
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)])
    except RuntimeError as e:
        print(e)

In [3]:
gpus

[]

# MS2

In [4]:
network = tf.keras.models.load_model('HCDSpectrumPredict/ucsd_hcd_splib_2ndhalf_model.h5')
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 30)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 512)         587776    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 512)         1574912   
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 4)           2052      
Total params: 2,164,740
Trainable params: 2,164,740
Non-trainable params: 0
_________________________________________________________________


In [5]:
data_fn = 'HCDSpectrumPredict/' + 'ucsd_hcd_splib_2ndhalf' + '.pickle'
with open(data_fn, 'rb') as f:
    zfit_reg_data = pickle.load(f)
training_X = zfit_reg_data['train_X']
training_y = zfit_reg_data['train_y']
testing_X = zfit_reg_data['test_X']
testing_y = zfit_reg_data['test_y']
print(training_X.shape, training_y.shape, testing_X.shape, testing_y.shape)
for _ in training_y[:5]:
    print(_)

(967409, 40, 30) (967409, 40, 4) (107490, 40, 30) (107490, 40, 4)
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.18748496 0.         0.52518387]
 [0.         0.13076325 1.         0.73639633]
 [0.         0.03434321 0.09965717 0.07823455]
 [0.         0.         0.         0.        ]
 [0.         0.         0.42424646 0.0726

In [8]:
network = models.Sequential()
network.add(layers.Masking(mask_value=0., input_shape=(None, training_X.shape[2])))
network.add(layers.Bidirectional(layers.LSTM(256,
                                 return_sequences=True))) 

network.add(layers.Bidirectional(layers.LSTM(256,
                                 return_sequences=True)))

network.add(layers.TimeDistributed(layers.Dense(4, activation='sigmoid')))
print(network.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, None, 30)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 512)         587776    
_________________________________________________________________
bidirectional_5 (Bidirection (None, None, 512)         1574912   
_________________________________________________________________
time_distributed (TimeDistri (None, None, 4)           2052      
Total params: 2,164,740
Trainable params: 2,164,740
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
network.compile(optimizer='adam', loss='mean_squared_error')
history = network.fit(training_X[:100], 
                  training_y[:100],
                    batch_size = 8, 
                  epochs=20)

cost = network.evaluate(testing_X, testing_y)
print('test cost:', cost)

Train on 100 samples
Epoch 1/20
100/100 [==============================] - 17s 172ms/sample - loss: 0.0344
Epoch 2/20
100/100 [==============================] - 3s 26ms/sample - loss: 0.0133
Epoch 3/20
100/100 [==============================] - 3s 27ms/sample - loss: 0.0127
Epoch 4/20
100/100 [==============================] - 3s 30ms/sample - loss: 0.0125
Epoch 5/20
100/100 [==============================] - 3s 28ms/sample - loss: 0.0123
Epoch 6/20
100/100 [==============================] - 3s 31ms/sample - loss: 0.0122
Epoch 7/20
100/100 [==============================] - 4s 35ms/sample - loss: 0.0119
Epoch 8/20
100/100 [==============================] - 3s 33ms/sample - loss: 0.0117
Epoch 9/20
100/100 [==============================] - 3s 31ms/sample - loss: 0.0113
Epoch 10/20
100/100 [==============================] - 3s 27ms/sample - loss: 0.0113
Epoch 11/20
100/100 [==============================] - 3s 28ms/sample - loss: 0.0115
Epoch 12/20
 96/100 [===========================>..

KeyboardInterrupt: 

In [6]:
network.predict(testing_X[:1])

array([[[0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.07488564, 0.32139403, 0.14188215, 0.14483967],
        [0.0382258 , 0.21027315, 0.10248446, 0.10217905],
        [0.01772472, 0.12465158, 0.08667448, 0.06394386],
        [0.00879481, 0.07558763, 0.08754134, 0.03867015],
        [0.00510207, 0.05057728, 0.09722361, 0.02438873],
        [0.00377512, 0.03910199, 0.11305165, 0.01753956],
        [0.00310659, 0.03261471, 0.12391868, 0.01411921],
        [0.00277606, 0.02947798, 0.13601294, 0.01215339],
        [0.00256965, 0.02736732, 0.14591688, 0.0111362 ],
        [0.002

# iRT

In [7]:
network = tf.keras.models.load_model('iRT/irt_reg_data_filtered_bidirLSTM2_masking_model.h5')
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 22)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 512)         571392    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 2,285,889
Trainable params: 2,285,889
Non-trainable params: 0
____________________________________________

In [8]:
scale_factor = 30.
data_fn = 'iRT/' + 'irt_reg_data_filtered' + '.pickle'
with open(data_fn, 'rb') as f:
    irt_reg_data = pickle.load(f)
training_X = irt_reg_data['training_X']
training_y = irt_reg_data['training_y']/scale_factor
testing_X = irt_reg_data['testing_X']
testing_y = irt_reg_data['testing_y']/scale_factor
print(training_X.shape, training_y.shape, testing_X.shape, testing_y.shape)
for _ in training_y[:5]:
    print(_)

(113215, 41, 22) (113215,) (12578, 41, 22) (12578,)
-0.543646704
3.671484405
1.2738423828333334
0.08875758625
4.4721045649999995


# ChargeState

In [9]:
network = tf.keras.models.load_model('ChargeState/zfit_bidirLSTM2_masking_model.h5')
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 23)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 512)         573440    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 2,280,965
Trainable params: 2,280,965
Non-trainable params: 0
_________________________________________________________________


In [ ]:
data_type = 'zfit'
scale_factor = 1.
data_fn = 'ChargeState/' + data_type + '_one_hot' + '.pickle'
with open(data_fn, 'rb') as f:
    zfit_reg_data = pickle.load(f)
training_X = zfit_reg_data['train_X']
training_y = zfit_reg_data['train_y']/scale_factor
testing_X = zfit_reg_data['test_X']
testing_y = zfit_reg_data['test_y']/scale_factor
print(training_X.shape, training_y.shape, testing_X.shape, testing_y.shape)
for _ in training_y[:5]:
    print(_)

In [ ]:
training_X

# data

# MS2

In [3]:
data_fn = 'HCDSpectrumPredict/' + 'ucsd_hcd_splib_2ndhalf' + '.pickle'
with open(data_fn, 'rb') as f:
    zfit_reg_data = pickle.load(f)
training_X = zfit_reg_data['train_X']
training_y = zfit_reg_data['train_y']
testing_X = zfit_reg_data['test_X']
testing_y = zfit_reg_data['test_y']
print(training_X.shape, training_y.shape, testing_X.shape, testing_y.shape)

(967409, 40, 30) (967409, 40, 4) (107490, 40, 30) (107490, 40, 4)


# iRT

In [15]:
data_fn = 'iRT/' + 'irt_reg_data_filtered' + '.pickle'
with open(data_fn, 'rb') as f:
    irt_reg_data = pickle.load(f)
print(irt_reg_data.keys())
training_X = irt_reg_data['training_X']
training_y = irt_reg_data['training_y']  # 나중에 사용할때는 scaling factor 찾아서 처리하도록.
testing_X = irt_reg_data['testing_X']
testing_y = irt_reg_data['testing_y']
print(training_X.shape, training_y.shape, testing_X.shape, testing_y.shape)

dict_keys(['training_X', 'training_y', 'testing_X', 'testing_y'])
(113215, 41, 22) (113215,) (12578, 41, 22) (12578,)


# Charge State

In [11]:
data_fn = 'ChargeState/' + 'zfit_one_hot' + '.pickle'
with open(data_fn, 'rb') as f:
    zfit_reg_data = pickle.load(f)
print(zfit_reg_data.keys())

training_X = zfit_reg_data['train_X']
training_y = zfit_reg_data['train_y']
testing_X = zfit_reg_data['test_X']
testing_y = zfit_reg_data['test_y']
print(training_X.shape, training_y.shape, testing_X.shape, testing_y.shape)

dict_keys(['train_X', 'train_y', 'test_X', 'test_y'])
(114188, 41, 23) (114188, 5) (12688, 41, 23) (12688, 5)
